<a href="https://colab.research.google.com/github/zahraDehghanian97/Poetry_Generator/blob/master/Word_Poem_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pickle
from nltk.metrics import accuracy ,ConfusionMatrix 
from nltk.translate.bleu_score import sentence_bleu

In [11]:
seqLength = 20
BATCH_SIZE = 64
BUFFER_SIZE = 100
embedding_dim = 256
rnn_units = 1024

# make data ready

In [12]:
filepath = "/content/drive/MyDrive/Colab Notebooks/my_shahname_represntation.txt"
with open(filepath, "rb") as f:
    corpus , test = pickle.load(f)
corpus = corpus.replace("\t"," \t ").replace("\n", " \n ")

In [13]:
corpusList = [w for w in corpus.split(' ')] 
corpus_words = [i for i in corpusList if i]
map(str.strip, corpus_words)
vocab = sorted(set(corpus_words))
print(len(corpus_words))
vocab_size = len(vocab)
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in corpus_words])

606052


In [14]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

# examples_per_epoch = len(corpus_words)//(seqLength + 1)
wordDataset = tf.data.Dataset.from_tensor_slices(word_as_int)
sequencesOfWords = wordDataset.batch(seqLength + 1, drop_remainder=True)
dataset = sequencesOfWords.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# LSTM Model

In [15]:
def create_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model

In [16]:
lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = lstm_model.fit(dataset, epochs=50)
main_lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=1)
main_lstm_model.set_weights(lstm_model.get_weights())
# main_lstm_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/word_lstm.h5')
main_lstm_model.summary()

Epoch 1/50
450/450 [==============================] - 64s 135ms/step - loss: 6.9648 - accuracy: 0.0743
Epoch 2/50
450/450 [==============================] - 61s 135ms/step - loss: 6.2403 - accuracy: 0.1036
Epoch 3/50
450/450 [==============================] - 61s 136ms/step - loss: 5.4921 - accuracy: 0.1513
Epoch 4/50
450/450 [==============================] - 61s 135ms/step - loss: 5.1126 - accuracy: 0.2139
Epoch 5/50
450/450 [==============================] - 61s 135ms/step - loss: 4.9087 - accuracy: 0.2264
Epoch 6/50
450/450 [==============================] - 61s 135ms/step - loss: 4.7511 - accuracy: 0.2362
Epoch 7/50
450/450 [==============================] - 61s 136ms/step - loss: 4.6100 - accuracy: 0.2447
Epoch 8/50
450/450 [==============================] - 61s 135ms/step - loss: 4.4782 - accuracy: 0.2521
Epoch 9/50
450/450 [==============================] - 61s 136ms/step - loss: 4.3420 - accuracy: 0.2596
Epoch 10/50
450/450 [==============================] - 61s 136ms/step - l

In [23]:
def generate_text(model, start_string):
  num_generate = 200
  start_string_list =[]
  for w in start_string.split(' '):
    if w in word2idx :
      start_string_list.append(w)
  input_eval = [word2idx[s] for s in start_string_list]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2words[predicted_id])
  return (start_string + ' '.join(text_generated))

In [21]:
print(generate_text(main_lstm_model, start_string=u"چنین گفت رستم به اسفندیار"))
main_lstm_model.save("/content/drive/MyDrive/Colab Notebooks/word_lstm.h5")

چنین گفت رستم به اسفندیارپیش اندرون 	 یکی خوب چیزی ندارد شتاب 
 ز من خواستم زور دست 
 که ما را بدی پاک ویران که در گنج دیگر شنید 
 به سی و گرازان بر اسپ آورید 	 به گرد اندرش آبهای شده 	 نبودش بهی شیر را آفرین 
 گزین کرد صندوق بر سنگ روی 	 کمان دل و زخم بربط بدند 
 دلیران برو چاک زد تاختن 	 به شنگل بفرمود کان سر سرکشان 
 چو بشنید سیندخت بر زال سام 	 نیایش کنان برگفتند راه 
 چو دیدند پیران به گشتاسپ گفت 	 که گفتار او باب دل خسته شد 
 دگر آنک دیدم ز ایران کنون 	 سزد گر به رزم اندرون نیست باک 
 چو اغریرث پرهنر شهریار 	 درختی همی گشت بر چرخ ماه 
 سر اندریمان به خاک آورید 	 دل جنگجویان سر اندر مبند 
 نخست آفرین کرد بر کردگار 	 و بخشایشست 	 برافروخت چون شیر فرمانروا 
 سه هر کس که گفتی برآرد ز درد 	 تن آسانی ما نگردد کهن 
 یکی بنده‌ام بد بخنجر همه 	 شده روز ایران به مازندران 
 بدین است کردار پیل و جوان 	 ز خویش و ز لشکر بباید بدر 
 بدان ای یکی نو


# Test

In [24]:
BLEU_scores = []
accuracy_scores = []
poem = test[0]
start = poem[:25]
generated_poem = generate_text(main_lstm_model, start_string=start)
BLEU_scores.append(sentence_bleu(poem, generated_poem))
len_min = min(len(poem),len(generated_poem))
accuracy_scores.append(accuracy(poem[:len_min], generated_poem[:len_min]))
print("-----------------------")
print("start sentence : ",start)
print(generated_poem)
print("BLEU score = ",BLEU_scores[-1])
print("Accuracy score = ",accuracy_scores[-1])
print("Confusion matrix =")
print(ConfusionMatrix(poem[:len_min], generated_poem[:len_min]))

counter = 0
for poem in test :
  counter+=1
  start = poem[:25]
  generated_poem = generate_text(main_lstm_model, start_string=start)
  BLEU_scores.append(sentence_bleu(poem, generated_poem))
  len_min = min(len(poem),len(generated_poem))
  accuracy_scores.append(accuracy(poem[:len_min], generated_poem[:len_min]))
  print("-----------------------")
  print("sentence number : ",counter)
  print("BLEU score = ",BLEU_scores[-1])
  print("Accuracy score = ",accuracy_scores[-1])

print("<<------------final report----------->>")
print("number of test set = ",len(test))
print("mean BLEU score = ",np.mean(BLEU_scores))
print("mean Accuracy score = ",np.mean(accuracy_scores))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


-----------------------
start sentence :  چو لشکر شد از خواسته بی‌ن
چو لشکر شد از خواسته بی‌نخواندمش 	 درم را بریزی به درویش بود 
 وگر دیر بودش پلنگ 
 برهنه شود ژنده او بر زمین 	 همی بفگند بر سرش آفتاب 
 شب تیره در درد چندان بدی 	 چو ابر سیاه 	 همی بود لرزان نیابد جواز 
 بباید کشیدن گمان از کسی کو بدانش سزد 
 تو دانی من این تخت را بر خورم 	 بدرد کسی زان توانگر چراست 
 بدو گفت نزدیک من بیش کیست 	 اگر تاج یابد هنر داد و خوار 
 نژند آیدم 	 نه از تیغ رنگ رخش ناپدید 
 فرود آمد از دشت نیزه‌وران 	 به لشکر به دشت اندر آمد سترگ 
 همی خواست تا شب گذشته سه پاس 
 یکی بخت صد دیگر آتش پر از خون و روی زرد برسان نیل 
 چه مردی بدو پهلوان خواندند 	 زبرجد ز کافور وز مشک شد ناپدید 
 سراپردهٔ دیبهٔ هفت‌رنگ 	 بپوشید در بیشه رخسار او 
 به کشتی کسی تا نگردد تهی 	 چو شد سال بر کشته از دست او 
 برو پیش بردند ایرانیان 	 چه بر تنش گیتی چه جویی همی 
 شما هم کنون از چه گوید سخن 	 بجز تو ران رنج افراسیاب 
 ازو برده افگنده بیکار
BLEU score =  0.43803861848327813
Accuracy score =  0.0997624703087886
Confusion matrix =